In [1]:
import numpy as np
from google.cloud import storage
from tempfile import TemporaryFile

In [3]:

rng = np.random.default_rng(2)
X = rng.random(400).reshape(-1,2)
X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
Y = np.zeros(len(X))
    
i=0
for t,d in X:
    y = -3/(260-175)*t + 21
    if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
        Y[i] = 1
    else:
        Y[i] = 0
    i += 1

#X, Y.reshape(-1,1)
Xt=np.tile(X,(1000,1))
Yt=np.tile(Y,(1000,1))
data_train=np.hstack((Xt, Yt.reshape(-1,1)))

In [4]:
storage_client = storage.Client()
bucket = storage_client.bucket('coffee_data_harveen')
blob_train = bucket.blob('orig_data/coffeeData.csv')

In [5]:
with TemporaryFile() as temp_file_train:
            np.savetxt(temp_file_train, data_train, delimiter=",",fmt='%f')
            temp_file_train.seek(0)
            blob_train.upload_from_file(temp_file_train)